In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy
import sklearn
from sklearn import linear_model
from sklearn import neighbors
from sklearn import ensemble
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import statsmodels.formula.api as smf
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [15]:
# Replace the path with the correct path for your data.
y2015 = pd.read_csv("LoanStats3d.csv",
                   skipinitialspace=True,
                   header=1)

# Note the warning about dtypes.

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
y2015.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,23850.0,23850.0,23850.0,60 months,17.27%,596.21,D,D3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,15000.0,15000.0,15000.0,60 months,10.78%,324.50,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,7200.0,7200.0,7200.0,36 months,17.27%,257.67,D,D3,...,NaN,NaN,Cash,Y,May-2018,BROKEN,Nov-2017,2761.0,50.01,10.0
3,NaN,NaN,12000.0,12000.0,12000.0,36 months,9.17%,382.55,B,B2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,11950.0,11950.0,11950.0,36 months,13.44%,405.18,C,C3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']

In [18]:
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

id
2
term
2
int_rate
111
grade
7
sub_grade
35
emp_title
120812
emp_length
11
home_ownership
4
verification_status
3
issue_d
12
loan_status
7
pymnt_plan
2
desc
34
purpose
14
title
27
zip_code
914
addr_state
49
earliest_cr_line
668
revol_util
1211
initial_list_status
2
last_pymnt_d
48
next_pymnt_d
6
last_credit_pull_d
49
application_type
2
verification_status_joint
1
hardship_flag
2
hardship_type
1
hardship_reason
9
hardship_status
3
hardship_start_date
23
hardship_end_date
24
payment_plan_start_date
24
hardship_loan_status
4
disbursement_method
1
debt_settlement_flag
2
debt_settlement_flag_date
41
settlement_status
3
settlement_date
44


In [19]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables.
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
           'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

In [20]:
y2015.tail()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
421092,NaN,NaN,13000.0,13000.0,13000.0,60 months,15.99,316.07,D,5 years,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
421093,NaN,NaN,12000.0,12000.0,12000.0,60 months,19.99,317.86,E,1 year,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
421094,NaN,NaN,20000.0,20000.0,20000.0,36 months,11.99,664.20,B,10+ years,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
421095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# Remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

In [22]:
pd.get_dummies(y2015).head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,settlement_date_Nov-2017,settlement_date_Nov-2018,settlement_date_Oct-2015,settlement_date_Oct-2016,settlement_date_Oct-2017,settlement_date_Oct-2018,settlement_date_Sep-2015,settlement_date_Sep-2016,settlement_date_Sep-2017,settlement_date_Sep-2018
0,NaN,NaN,23850.0,23850.0,23850.0,17.27,596.21,68046.0,24.71,1.0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,NaN,15000.0,15000.0,15000.0,10.78,324.50,65000.0,29.28,0.0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,NaN,7200.0,7200.0,7200.0,17.27,257.67,20000.0,13.21,1.0,...,1,0,0,0,0,0,0,0,0,0
3,NaN,NaN,12000.0,12000.0,12000.0,9.17,382.55,39400.0,26.32,0.0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,NaN,11950.0,11950.0,11950.0,13.44,405.18,34000.0,10.20,0.0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


array([0.9868446 , 0.98544358, 0.98805566, 0.98955142, 0.98831631,
       0.98909998, 0.98883796, 0.98945543, 0.99002541, 0.98952668])

## DRILL: Third Attempt

So here's your task. Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

You'll want to do a few things in this process. First, dive into the data that we have and see which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matrices.

Can you do it without using anything related to payment amount or outstanding principal? How do you know?

In [24]:
# Let's use the Random Forest Classifier to rank feature importance.
rfc = ensemble.RandomForestClassifier()
rfc.fit(X, Y)
names_importances = {}

for feature, importance in zip(X.columns, rfc.feature_importances_):
    names_importances[feature] = importance
    
names_importances_desc = sorted(names_importances.items(), key=lambda x: x[1], reverse=True)
names_importances_desc = pd.DataFrame(names_importances_desc, columns=['feature', 'importance'])

In [28]:
names_importances_desc.head(20)

,feature,importance
0,collection_recovery_fee,0.133575
1,last_pymnt_amnt,0.096334
2,recoveries,0.092860
3,out_prncp_inv,0.092771
4,last_pymnt_d_Dec-2018,0.085807
5,total_rec_prncp,0.075772
6,next_pymnt_d_Jan-2019,0.064227
7,term_ 60 months,0.043127
8,term_ 36 months,0.037812
9,total_pymnt_inv,0.033203


Most of these columns deal with either payment amount or outstanding principal.  Let's drop a few of these columns and see how it changes our features.

In [29]:
X_2 = X.drop(['last_pymnt_amnt',
              'out_prncp_inv',
              'last_pymnt_d_Dec-2018',
              'total_rec_prncp',
              'next_pymnt_d_Jan-2019',
              'total_pymnt_inv',
              'total_pymnt',
              'funded_amnt',
              'funded_amnt_inv',
              'out_prncp',],1)
X_2 = X_2.dropna(axis=1)
cross_val_score(rfc, X_2, Y, cv=10)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


array([0.7993446 , 0.90233188, 0.92194624, 0.91717129, 0.93189266,
       0.93664213, 0.92542808, 0.89234569, 0.93364524, 0.93535517])

In [30]:
X_prime = X[['collection_recovery_fee',
             'recoveries',
             'term_ 60 months',
             'term_ 36 months',
             'loan_amnt',
             'last_credit_pull_d_Dec-2018',
             'installment',
             'settlement_status_ACTIVE',
             'grade_A']]
features = list(X_prime.columns)

for i in range(len(features), 0, -1):
    X = X[features[:i+1]]
    print('Run Number: {}.\n'.format(len(features)-i+1))
    print('Number of Features: {}.\n'.format(i))
    print('Dropped Feature(s): \n{}\n'.format(features[i:]))
    print('Average Accuracy (10-Fold Cross-Validation): {:.2%}.\n'.format(cross_val_score(rfc, X_prime, Y, cv=10).mean()))

Run Number: 1.

Number of Features: 9.

Dropped Feature(s): 
[]



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


Average Accuracy (10-Fold Cross-Validation): 86.58%.

Run Number: 2.

Number of Features: 8.

Dropped Feature(s): 
['grade_A']



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


Average Accuracy (10-Fold Cross-Validation): 86.62%.

Run Number: 3.

Number of Features: 7.

Dropped Feature(s): 
['settlement_status_ACTIVE', 'grade_A']



...

Average Accuracy (10-Fold Cross-Validation): 86.60%.

Run Number: 8.

Number of Features: 2.

Dropped Feature(s): 
['term_ 60 months', 'term_ 36 months', 'loan_amnt', 'last_credit_pull_d_Dec-2018', 'installment', 'settlement_status_ACTIVE', 'grade_A']



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


Average Accuracy (10-Fold Cross-Validation): 86.61%.

Run Number: 9.

Number of Features: 1.

Dropped Feature(s): 
['recoveries', 'term_ 60 months', 'term_ 36 months', 'loan_amnt', 'last_credit_pull_d_Dec-2018', 'installment', 'settlement_status_ACTIVE', 'grade_A']



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


Average Accuracy (10-Fold Cross-Validation): 86.57%.

